# Demo: Loading Server-Side Files into Memory

### 1. Import Packages and Connect to the CAS Server
Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [ ]:
## Import packages
import swat
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Set options
pd.set_option('display.max_columns', None)

## Connect to CAS
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0', name = 'py02d02')

### 2. Explore Available Caslibs and Files


a. Use the caslibInfo action to display a list of available caslibs.

In [ ]:
conn.caslibInfo()

b. Use the tableInfo action to display available CAS tables in the **Casuser** caslib.

In [ ]:
conn.tableInfo(caslib = 'casuser')

c. Use the fileInfo action to display available files in the **PIVY** caslib. 

In [ ]:
conn.fileInfo(caslib = 'PIVY')

### 3. Loading Server-Side Files into Memory

#### Loading the cars.sas7bdat File into Memory

a. Use the [table.loadTable](https://go.documentation.sas.com/doc/en/pgmsascdc/v_017/caspg/cas-table-loadtable.htm) action to load the **cars.sas7bdat** file into memory from the **Casuser** caslib. The path parameter specifies the **cars.sas7bdat** data source file to load, and the caslib parameter specifies the input **Casuser** caslib. 

The casOut parameter specifies the output table information. Inside the casOut parameter, 
- the name sub-parameter names the new CAS table **CARS_SAS7BDAT**, 
- the caslib sub-parameter specifies the **Casuser** caslib as the output caslib, 
- the replace sub-parameter overwrites the CAS table if it already exists. 

**Note:** The action returns a **CASResults** object (dictionary) of information to the client about the load into memory.

In [ ]:
conn.loadTable(path = 'cars.sas7bdat', caslib = 'PIVY',
               casOut={'name' : 'cars_sas7bdat', 
                       'caslib' : 'casuser', 
                       'replace' : True})

b. Use the tableInfo action to view the available CAS tables. Notice the **CARS_SAS7BDAT** table is now available.

In [ ]:
conn.tableInfo(caslib = 'casuser')

c. After the table is loaded into memory, you can reference the CAS table **CARS_SAS7BDAT** in the variable **cars** using the CASTable method. Then use the head method on the **cars** variable to preview the table.

In [ ]:
cars = conn.CASTable('cars_sas7bdat', caslib = 'casuser')
cars.head()

d. In the loadTable action add, the where and vars parameters to subset the original data source file when loading it into memory. The where parameter specifies all rows where **Origin** equals *Europe*, and the vars parameter specifies only the columns **Make**, **Model**, **Origin**, and **MSRP**. Then the casOut parameter is used to specify output CAS table information. Then use the tableInfo action to view the available in-memory tables. Execute the cell. Notice that the new  **CARS_SUBSET** CAS table contains 123 rows and 4 columns.

In [ ]:
conn.loadTable(path = 'cars.sas7bdat', caslib = 'PIVY',
               where = 'Origin = "Europe"',
               vars = ['Make','Model','Origin','MSRP'],
               casOut = {'name' : 'cars_subset', 
                         'caslib' : 'casuser', 
                         'replace' : True})

conn.tableInfo(caslib = 'casuser')

#### Loading the customers_raw.csv File into Memory

a. Execute the fileInfo action to view available data source files. Notice that the **customers_raw.csv** file exists in the **PIVY** caslib.

In [ ]:
conn.fileInfo(caslib = 'PIVY')

b. The loadTable action can be used to load all types of data source files into memory. Here, the loadTable action loads the **customers_raw.csv** file into memory and names the new CAS table **CUSTOMERS_RAW**.

In [ ]:
conn.loadTable(path = 'customers_raw.csv', caslib = 'PIVY',
               casOut = {'name' : 'customers_raw', 
                         'caslib' : 'casuser', 
                         'replace' : True})

c. Use the tableInfo action to view the available CAS tables. Notice that the **CUSTOMERS_RAW** table is now available.

In [ ]:
conn.tableInfo(caslib = 'casuser')

d. After the table is loaded into memory, you can reference the **CUSTOMERS_RAW** CAS table and begin to process the data. First, the CASTable method references the CAS table in the variable **customers**. Then, the SWAT [query](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.query.html#swat.cas.table.CASTable.query) method is used to subset the **CUSTOMERS_RAW**  table where the **Country** value is *CA*, and then the head method is used to return five rows to the client.

In [ ]:
customers = conn.CASTable('CUSTOMERS_RAW', caslib = 'casuser')

(customers
 .query('Country = "CA"')
 .head())

e. For additional import options, you can use the [importOptions](https://go.documentation.sas.com/doc/en/pgmsascdc/v_017/caspg/compg-importoptions-114param.htm) parameter. In the importOptions parameter, you specify the fileType sub-parameter with the file type that you want to import. The fileType value specifies the settings available for reading a table from a data source.Here, fileType specifies a CSV file, then the vars sub-parameter is used to name the columns using the **colNames** variable, and the guessRows equals 100 sub-parameter to scans the first 100 rows of each column to determine the columns data type.

In [ ]:
## Specify a list of column names
colNames = ['CustID_rename', 'CustAge_rename', 'CustSalary_rename', 'CustEmpLength', 'Date', 'Email', 'Address',
            'Country', 'LoanCreditCard', 'LoanConsolidation', 'LoanMortgage',
            'LoanHomeImprovement', 'LoanCarLoan', 'LoanPersonal',
            'LoanMovingExpenses', 'LoanSmallBusiness', 'LoanVacation',
            'LoanMedical', 'LoanMajorPurchase', 'LoanWeddings', 'LoanEducation',
            'SavingsAcct', 'CheckingAcct']

## Load the CSV file using the importOptions parameter to modify column names
conn.loadTable(path = 'customers_raw.csv', caslib = 'PIVY',
               casOut = {'name' : 'customers_raw_fileType', 
                         'caslib' : 'casuser', 
                         'replace' : True},
               importOptions = {'fileType' : 'CSV', 
                                'vars' : colNames,
                                'guessRows' : 100})


## View available CAS table
conn.tableInfo(caslib = 'casuser')

f. Reference the new **CUSTOMERS_RAW_FILETYPE** CAS table in the variable **c**. Then view the newly renamed column names by executing the head method on the CAS table to retrieve five rows to the client, and then the loc method on the **SASDataFrame** to specify the columns.

In [ ]:
c = conn.CASTable('customers_raw_fileType', caslib = 'casuser');

(c
 .head()
 .loc[:, ['CustID_rename', 'CustAge_rename', 'CustSalary_rename']])

### 4. Use the load_path Method

a. You can also use the [load_path](https://sassoftware.github.io/python-swat/generated/swat.cas.connection.CAS.load_path.html#swat.CAS.load_path) method to load server-side files into memory. This is a convenience  method. It executes the loadTable action and references the CAS table in one step. Here, the load_path method loads the **customers_raw.csv** file into memory and then references the CAS table in the variable **CUSTOMERS2**. Once the table is referenced you can execute the fetch action to return ten rows to the client.

**Note:** In comparison, the loadTable action returns a **CASResults** object to the client with information about the loaded table. You then have to reference the table manually.

In [ ]:
customers2 = conn.load_path(path = 'customers_raw.csv', caslib = 'PIVY',
                            casOut = {'name' : 'customers_raw_load_path', 
                                      'caslib' : 'casuser', 
                                      'replace' : True},
                            importOptions = {'fileType' : 'CSV', 
                                             'guessRows' : 100},
                            where = 'Age < 20',
                            vars = ['ID','Age','Salary'])

customers2.fetch(to = 10)

### 5. Terminate the CAS Session

It's best practice to always terminate the CAS session when you are done.

In [ ]:
conn.terminate()